# Data Exploration using pyspark and pandas

* We are going to explore the movies_metadata.csv file from the TMDB dataset. 
* We will primarily utilise pyspark and pandas for this
* For plotting we'll use seaborn and matplotlib

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Starting a spark session locally
spark = SparkSession \
    .builder \
    .appName("movie-rec-als") \
    .getOrCreate()

In [29]:
# Let's read in the movie metadata and see what it contains
movie_met = spark.read.csv('data/movies_metadata.csv', header=True)
movie_met.printSchema()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

In [30]:
# We see plenty of columns that we do not require in out analysis. We'll drop them
# Also notice that the dtype is string for all columns
# This might imply that they contain a mix of different dytpes

# Dropping the columns not needed
movie_met = movie_met.drop(*['homepage', 'imdb_id', 'original_title', 'poster_path', 'video', 'belongs_to_collection', 'revenue', 'tagline', 'overview', 'spoken_languages', 'production_countries'])

In [31]:
movie_met.toPandas().describe(include='all')

,adult,budget,genres,id,original_language,popularity,production_companies,release_date,runtime,status,title,vote_average,vote_count
count,45572,45555,45549,45541,45527,45452,45448,45378,45168,45327,44794,45074,45183
unique,111,1358,4172,45469,246,43911,24644,19065,2535,1367,40102,1573,2898
top,False,0,"[{'id': 18, 'name': 'Drama'}]","[{'id': 35, 'name': 'Comedy'}]",en,0.0,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",90.0,Released,Released,0.0,1
freq,45454,36509,4996,15,32185,67,10871,478,2334,41194,749,2756,2972


In [6]:
def count_nans(df):
    return df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).toPandas()
    
def count_nulls(df):
    return df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas()

In [7]:
count_nulls(movie_met)

,adult,budget,genres,id,original_language,popularity,production_companies,release_date,runtime,status,title,vote_average,vote_count
0,0,17,23,31,45,120,124,194,404,245,778,498,389


In [8]:
count_nans(movie_met)

,adult,budget,genres,id,original_language,popularity,production_companies,release_date,runtime,status,title,vote_average,vote_count
0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
def to_dtype_remove_null(df, col_name, dtype_out):
    df = df.withColumn(col_name, col(col_name).cast(dtype_out))
    df = df.filter(df[col_name].isNotNull())
    return df

In [40]:
# Cleaning the id, popularity, runtime, vote_average, vote_count columns
# Changing to type int and removing Nan/null rows

movie_met = to_dtype_remove_null(movie_met, 'id', 'int')
movie_met = to_dtype_remove_null(movie_met, 'popularity', 'int')
movie_met = to_dtype_remove_null(movie_met, 'runtime', 'int')
movie_met = to_dtype_remove_null(movie_met, 'vote_average', 'int')
movie_met = to_dtype_remove_null(movie_met, 'vote_count', 'int')
movie_met = to_dtype_remove_null(movie_met, 'budget', 'int')

In [35]:
# Keeping on the release year
# Cleaning NaN/Null rows

movie_met = movie_met.withColumn('year', year(movie_met['release_date'])).drop('release_date')
movie_met = movie_met.filter(movie_met['year'].isNotNull())

In [36]:
count_nulls(movie_met)

,adult,budget,genres,id,original_language,popularity,production_companies,runtime,status,title,vote_average,vote_count,year
0,0,0,0,0,11,0,0,0,65,0,0,0,0


In [24]:
movie_met.filter(movie_met['original_language'].isNull()).toPandas().head()

,adult,budget,genres,id,original_language,popularity,production_companies,runtime,status,title,vote_average,vote_count,year
0,False,0,"[{'id': 99, 'name': 'Documentary'}]",283101,None,0,"[{'name': 'StudioCanal', 'id': 694}, {'name': ...",95,Released,Shadowing the Third Man,0,0,2004
1,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",103902,None,0,"[{'name': 'New Holland Pictures', 'id': 10229}...",94,Released,Unfinished Sky,6,8,2007
2,False,0,"[{'id': 10752, 'name': 'War'}, {'id': 37, 'nam...",359195,None,0,"[{'name': 'Associated Producers (API)', 'id': ...",69,Released,13 Fighting Men,0,0,1960
3,False,0,"[{'id': 35, 'name': 'Comedy'}]",147050,None,0,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",8,Released,Lambchops,6,2,1929
4,False,0,"[{'id': 16, 'name': 'Animation'}]",257095,None,0,"[{'name': 'Ceskoslovenský Státní Film', 'id': ...",87,Released,Prince Bayaya,5,1,1950


In [22]:
# Seeing how many movies are rated adult or otherwise
movie_met.groupBy('adult').count().orderBy('count', ascending=False).toPandas()

,adult,count
0,False,41106
1,True,8


In [25]:
# Seeing how many movies are released or otherwise
movie_met.groupBy('status').count().orderBy('count', ascending=False).toPandas()

,status,count
0,Released,40723
1,Rumored,205
2,Post Production,93
3,None,65
4,In Production,16
5,Planned,12


In [37]:
# Let's just keep the adult=False rows and drop this column.
# movie_met.filter(movie_met['adult'] == 'False').groupBy('adult').count().orderBy('count', ascending=False).toPandas()
movie_met = movie_met.filter(movie_met['adult'] == 'False').drop('adult')

In [38]:
 # Let's just keep the movies that are released.
movie_met = movie_met.filter(movie_met['status'] == 'Released').drop('status')

In [41]:
movie_met.printSchema()

root
 |-- budget: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- original_language: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: integer (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- year: integer (nullable = true)



In [42]:
movie_met.toPandas().describe(include='all')

,budget,genres,id,original_language,popularity,production_companies,runtime,title,vote_average,vote_count,year
count,4.071500e+04,40715,40715.000000,40705,40715.000000,40715,40715.000000,40715,40715.000000,40715.000000,40715.000000
unique,NaN,3839,NaN,86,NaN,20695,NaN,38023,NaN,NaN,NaN
top,NaN,"[{'id': 18, 'name': 'Drama'}]",NaN,en,NaN,[],NaN,Cinderella,NaN,NaN,NaN
freq,NaN,4507,NaN,29083,NaN,10354,NaN,11,NaN,NaN,NaN
mean,4.336222e+06,NaN,108111.174800,NaN,2.560088,NaN,94.374162,NaN,5.272037,111.998060,1991.963011
std,1.769652e+07,NaN,112364.707662,NaN,6.115530,NaN,38.316838,NaN,1.822373,490.444431,24.197578
min,0.000000e+00,NaN,2.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,0.000000,1874.000000
25%,0.000000e+00,NaN,26120.000000,NaN,0.000000,NaN,85.000000,NaN,5.000000,3.000000,1978.000000
50%,0.000000e+00,NaN,59852.000000,NaN,1.000000,NaN,95.000000,NaN,6.000000,10.000000,2001.000000
75%,0.000000e+00,NaN,157356.500000,NaN,3.000000,NaN,107.000000,NaN,6.000000,36.000000,2011.000000


In [45]:
# Checking for duplicate ids
movie_met.groupBy('id').count().orderBy('count', ascending=False).toPandas()

,id,count
0,141971,3
1,265189,2
2,109962,2
3,110428,2
4,25541,2
...,...,...
40684,201429,1
40685,131343,1
40686,119984,1
40687,194039,1


In [46]:
movie_met.filter(movie_met['id'] == 141971).toPandas()

,budget,genres,id,original_language,popularity,production_companies,runtime,title,vote_average,vote_count,year
0,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",141971,fi,0,"[{'name': 'Filmiteollisuus Fine', 'id': 5166}]",108,Blackout,6,3,2008
1,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",141971,fi,0,"[{'name': 'Filmiteollisuus Fine', 'id': 5166}]",108,Blackout,6,3,2008
2,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",141971,fi,0,"[{'name': 'Filmiteollisuus Fine', 'id': 5166}]",108,Blackout,6,3,2008


In [47]:
# There are a few duplicate ids and titles
# movie_met.dropDuplicates(['id']).groupBy('id').count().orderBy('count', ascending=False).toPandas()
movie_met = movie_met.dropDuplicates(['id'])

In [48]:
movie_met.groupBy('title').count().orderBy('count', ascending=False).toPandas()

,title,count
0,Cinderella,11
1,Alice in Wonderland,9
2,Beauty and the Beast,8
3,Hamlet,8
4,Les Misérables,7
...,...,...
38018,You Are God,1
38019,Life Stinks,1
38020,Barbie: A Fairy Secret,1
38021,Boca,1


In [49]:
movie_met.filter(movie_met['title'] == 'Cinderella').toPandas().head()

,budget,genres,id,original_language,popularity,production_companies,runtime,title,vote_average,vote_count,year
0,0,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 107...",42884,en,2,"[{'name': 'Walt Disney Television', 'id': 670}...",88,Cinderella,6,28,1997
1,0,[],289673,en,1,[],90,Cinderella,5,5,2000
2,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...",92349,en,0,[],52,Cinderella,5,7,1914
3,2900000,"[{'id': 10751, 'name': 'Family'}, {'id': 14, '...",11224,en,14,"[{'name': 'RKO Radio Pictures', 'id': 6}, {'na...",74,Cinderella,6,1760,1950
4,95000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",150689,en,19,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...",105,Cinderella,6,2426,2015


Appears that the duplicate movies are actually distinct. Let's just keep them

In [50]:
count_nulls(movie_met)

,budget,genres,id,original_language,popularity,production_companies,runtime,title,vote_average,vote_count,year
0,0,0,0,10,0,0,0,0,0,0,0
